# Download

Fetch, save and combine food inspection records [from Open Minneapolis](https://opendata.minneapolismn.gov/datasets/4eea8bf452e34f8c9d9ac07c54c0b4ab_0/about).

Import Python tools

In [47]:
import json
from typing import Any
from datetime import datetime

import requests
import pandas as pd
import geopandas as gpd

Put together a function that can fetch all of the records in the given year. This is necessary because of the 32,000 record count limit in the Open Minneapolis API. I'm betting that we'll never top that number if we segment by year.

In [56]:
def get_year(year: int) -> Any:
    # Set the URL
    url = "https://services.arcgis.com/afSMGVsC7QlRK1kZ/arcgis/rest/services/Food_Inspections/FeatureServer/0/query"

    # Make our request
    r = requests.get(
        url,
        params={
            # Set a date filter with the input year
            "where": f"DateOfInspection>'{year}-01-01' AND DateOfInspection < '{year}-12-31'",
            # The GeoJSON stuff
            "f": "geojson",
            "outSR": "4326",
            # Take all the fields
            "outFields": "*"
        }
    )
    # Make sure the response is okay
    assert r.ok
    # Get the data as JSON
    data = r.json()
    # Print the result
    print(f"{year}: {len(data['features'])} records")
    # Make sure we don't have exactly 32,000 records, which is a telltale sign we've topped out
    assert len(data['features']) != 32_000
    # Return the result
    return data

Go get every year up until the current day

In [57]:
year_list = list(range(2021, datetime.now().year + 1))

Loop through the years and write out all the responses as year stamped files.

In [58]:
for year in year_list:
    geojson = get_year(year)
    with open(f"{year}.geojson", "w") as fp:
        json.dump(geojson, fp)

2021: 7748 records
2022: 10609 records
2023: 13835 records
2024: 7084 records


Combine all of the files into a single dataframe

In [59]:
gdf_list = []
for year in year_list:
    gdf = gpd.read_file(f"{year}.geojson")
    gdf_list.append(gdf)

In [64]:
df = pd.concat(gdf_list)

Convert it into a GeoDataFrame

In [66]:
gdf = gpd.GeoDataFrame(df)

Write the result out to a combined file.

In [68]:
gdf.to_file("food-inspections.geojson", driver="GeoJSON")